In [1]:
import os
import numpy as np
import re
import pickle
import sys

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

from ProcessEventsFunctions import *
sys.path.insert(1, 'Old')
from Steef_Functions import *

In [2]:
home_dir = '/nfs/a319/gy17m2a/PhD/'
home_dir2 = '/nfs/a161/gy17m2a/PhD/'

In [3]:
quintile_mapping = {1: 'F2', 2: 'F1', 3: 'C', 4: 'B1', 5: 'B2'}
quintile_mapping_thirds = {1: 'F', 2: 'C', 3: 'B'}

In [4]:
tbo_vals = pd.read_csv(home_dir + 'datadir/RainGauge/interarrival_thresholds_CDD_noMissing.txt')
# Check if the points are within the areas
tbo_vals = check_for_gauge_in_areas(tbo_vals, home_dir, ['NW', 'NE', 'ME', 'SE', 'SW'])
tbo_vals.loc[tbo_vals['within_area'] == 'NW, C', 'within_area'] = 'NW'
tbo_vals.loc[tbo_vals['within_area'] == 'ME, SE', 'within_area'] = 'ME'

### Define ensemble members for present and future

In [5]:
ems_present = ['bc005', 'bc006', 'bc007', 'bc009', 'bc010', 'bc011', 'bc012', 'bc013', 'bc015', 'bc016', 'bc017', 'bc018']
ems_future = ['bb189']

### Get events (considering one set of AMAX producing events (with duplicates deleted)

In [6]:
# # Now you can call the function for both time periods
# events_dict_present, event_props_dict_present, event_profiles_dict_present = process_events_alltogether(home_dir2, 'Present',ems_present, tbo_vals, home_dir)
events_dict_future, event_props_dict_future, event_profiles_dict_future = process_events_alltogether(home_dir2, 'Future', ems_future, tbo_vals, home_dir)

bb189
Processing gauge 0


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_209078/231556802.py", line 3, in <module>
    events_dict_future, event_props_dict_future, event_profiles_dict_future = process_events_alltogether(home_dir2, 'Future', ems_future, tbo_vals, home_dir)
  File "/nfs/a319/gy17m2a/PhD/Scripts/FindIndependentRainfallEvents/ProcessEvents/ProcessEventsFunctions.py", line 112, in process_events_alltogether
    this_event = read_event(gauge_num, fp)
  File "/nfs/a319/gy17m2a/PhD/Scripts/FindIndependentRainfallEvents/ProcessEvents/ProcessEventsFunctions.py", line 76, in read_event
    test = pd.read_csv(fp)
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/pandas/util/_decorators.py", line 311, in wrapper
    return func(*args, **kwargs)
  File "

TypeError: object of type 'NoneType' has no len()

In [47]:
with open(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/Present/events_dict_present.pickle", 'wb') as handle:
    pickle.dump(events_dict_present, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/Future/events_dict_future.pickle", 'wb') as handle:
    pickle.dump(events_dict_future, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/Present/event_profiles_dict_present", 'wb') as handle:
    pickle.dump(event_profiles_dict_present, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/Future/event_profiles_dict_future.pickle", 'wb') as handle:
    pickle.dump(event_profiles_dict_future, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/Present/event_props_dict_present.pickle", 'wb') as handle:
    pickle.dump(event_props_dict_present, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/Future/event_props_dict_future.pickle", 'wb') as handle:
    pickle.dump(event_props_dict_future, handle, protocol=pickle.HIGHEST_PROTOCOL)    

## Get events for each duration (there'll be cross over in events present for each duration)
By duration here we mean the duration for which the AMAX are associated, rather than the actual duration of the event

In [ ]:
# List of desired durations
valid_durations = ["0.5", "1", "2", "3", "6", "12", "24"]

# Process events for both time periods
results_present = process_events_by_duration(home_dir2, 'Present', valid_durations, ems_present, tbo_vals)
# results_future = process_events_by_duration('Future', valid_durations, ems_future, tbo_vals)

In [18]:
dur_present_events_dict, dur_present_event_props_dict, dur_present_event_profiles_dict = results_present
# dur_future_events_dict, dur_future_event_props_dict, dur_future_event_profiles_dict = results_future

In [15]:
with open(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/Present/results_each_dur_present.pickle", 'wb') as handle:
    pickle.dump(results_present, handle, protocol=pickle.HIGHEST_PROTOCOL)    
# with open(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/Future/results_each_dur_future.pickle", 'wb') as handle:
#     pickle.dump(results_future, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Do we have the same number of results for each duration?
I believe that longer durations have more events because of compound annual maxima

In [19]:
for duration in valid_durations:
    print(duration, len(dur_present_event_props_dict[duration].keys()))

0.5 2964
1 2964
2 2964
3 2964
6 2966
12 3022
24 3449


In [33]:
# List of desired durations
duration_bins = ['<4hr', '4-12hr', '12hr+']

# Process events for both time periods
results_present_dur_categories_simple = process_events_by_duration(home_dir2, 'Present', duration_bins, ems_present, tbo_vals)
results_future_dur_categories_simple = process_events_by_duration(home_dir2, 'Future', duration_bins, ems_future, tbo_vals)

bc005
Processing gauge 0
Processing gauge 100
Processing gauge 200
Processing gauge 300
Processing gauge 400
Processing gauge 500
Processing gauge 600
Processing gauge 700
Processing gauge 800
Processing gauge 900
Processing gauge 1000
Processing gauge 1100
Processing gauge 1200
bc006
Processing gauge 0
Processing gauge 100
Processing gauge 200
Processing gauge 300
Processing gauge 400
Processing gauge 500
Processing gauge 600
Processing gauge 700
Processing gauge 800
Processing gauge 900
Processing gauge 1000
Processing gauge 1100
Processing gauge 1200
bc007
Processing gauge 0
Processing gauge 100
Processing gauge 200
Processing gauge 300
Processing gauge 400
Processing gauge 500
Processing gauge 600
Processing gauge 700
Processing gauge 800
Processing gauge 900
Processing gauge 1000
Processing gauge 1100
Processing gauge 1200
bc009
Processing gauge 0
Processing gauge 100
Processing gauge 200
Processing gauge 300
Processing gauge 400
Processing gauge 500
Processing gauge 600
Processin

In [36]:
with open(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/Present/results_present_dur_categories_simple.pickle", 'wb') as handle:
    pickle.dump(results_present_dur_categories_simple, handle, protocol=pickle.HIGHEST_PROTOCOL)    
with open(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/Future/results_future_dur_categories_simple.pickle", 'wb') as handle:
    pickle.dump(results_future_dur_categories_simple, handle, protocol=pickle.HIGHEST_PROTOCOL)    